In [24]:
import pandas as pd

# prepare dataframe

In [25]:
import io
df = pd.read_csv(r"data\d.csv", encoding="utf-8", encoding_errors='ignore')
# Dataset is now stored in a Pandas Dataframe

In [26]:
df.drop(['count', 'offensive_language','neither','hate_speech'], axis = 1, inplace = True)
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [27]:
df_first = df["class"].transform(lambda x: x.replace([1,2], "XX"))
df["class"] = df_first.transform(lambda x: x.replace(0,1).replace("XX",0)) #hate-speach = 1 in class everitying else  = 0

# split data

In [28]:
print(df['tweet'].to_string())

0        !!! RT @mayasolovely: As a woman you shouldn't...
1        !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3        !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
5        !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...
6        !!!!!!"@__BrighterDays: I can not just sit up ...
7        !!!!&#8220;@selfiequeenbri: cause I'm tired of...
8        " &amp; you might not get ya bitch back &amp; ...
9        " @rhythmixx_ :hobbies include: fighting Maria...
10       " Keeks is a bitch she curves everyone " lol I...
11                      " Murda Gang bitch its Gang Land "
12       " So hoes that smoke are losers ? " yea ... go...
13           " bad bitches is the only thing that i like "
14                                 " bitch get up off me "
15                         " bitch nigga miss me with it "
16                                  " bitch plz whatever

In [29]:
from sklearn.model_selection import train_test_split

X_data = df['tweet']
traget_class = df['class']


X_train, X_test , y_train, y_test = train_test_split(X_data, traget_class , test_size = 0.2)

In [30]:
import re 

 
def preprocessing(tweet):

  #use prepro of glove stanford 4 tweets https://nlp.stanford.edu/projects/glove/preprocess-twitter.rb

   

  #done  replace @user with "USERNAME1"
  # handel emoji 
  #done replace all links with "LINK1"
  # delete ! and points
  # lower
  # maybe stamming delete 

  re_username = '@[\w\-]+'
  re_exclamation = ""
  re_emoji ="" # delete or not
  re_link = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
  re_all_words = "[^a-z ]+"


  tweet=re.sub(re_username,"username", tweet)
  tweet=re.sub(re_link,"link", tweet)
  tweet = tweet.lower() # only lower characters maybe better maybe worse
  tweet = re.sub(re_all_words," ",tweet) #space beacuse otherwise u merge some words



  #read article if which stamming to use whit glove standford https://www.kaggle.com/general/218317
  tweet = tweet #stamming maybe better maybe worse

  

  return tweet

"import nltk\nfrom nltk.corpus import stopwords\nnltk.download('stopwords')\nnltk.download('punkt')\nfrom nltk.tokenize import word_tokenize\n\n \ndef tokenizer_word_tokenize(tweet):\n  tokens_tweet = word_tokenize(tweet)\n  tokens_without_sw = [word for word in tokens_tweet if not word in stopwords.words()]\n\n  return tokens_tweet"

In [32]:
# code for Glove word embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
  

    

def glove_embedings(x):

    #print("X----------->",x)

    x = {x}

    # create the dict.
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)

    #print(tokenizer.word_index)



    #print("Number of unique words in dictionary=", len(tokenizer.word_index))
    #print("Dictionary is = ", tokenizer.word_index)



    def embedding_for_vocab(filepath, word_index, embedding_dim):

        vocab_size = len(word_index) + 1
        
        # Adding again 1 because of reserved 0 index
        embedding_matrix_vocab = np.zeros((vocab_size, embedding_dim))
    
        with open(filepath, encoding="utf8") as f:
            for line in f:
                word, *vector = line.split()
                if word in word_index:
                    idx = word_index[word]
                    embedding_matrix_vocab[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]
    
        return embedding_matrix_vocab
    
    
    # matrix for vocab: word_index
    embedding_dim = 100
    embedding_matrix_vocab = embedding_for_vocab('data\glove.twitter.27B.100d.txt', tokenizer.word_index, embedding_dim)
    
    #print("Dense vector for first word is => ", len(embedding_matrix_vocab))
    #print(embedding_matrix_vocab[1])

    return embedding_matrix_vocab

In [33]:
def global_preprocessing(data):


    #handling emojis
    #df["tweet"] = df["tweet"].transform(lambda x: transform_emojis(x))
    
    
    data=data[:3] #df cutted for faster testing 
    print(type(data))
    print(data)


    #print(preprocessing("test"))

    data = data.transform(lambda x: preprocessing(x))
    print(type(data))
    print(data)

    # check if stopwords needet
    #data = data.transform(lambda x: tokenizer(x,df["class"])) #df cutted for faster testing 
    


    #df["tweet"] = df["tweet"][:500].transform(lambda x: tokenizer_word_tokenize(x)) #df cutted for faster testing 
    #df=df[:500] #df cutted for faster testing 

    data = data.transform(lambda x: glove_embedings(x))


    return data

In [34]:
data = global_preprocessing(X_train)

<class 'pandas.core.series.Series'>
11696    If your twitter is private but your Instagram ...
4507     @RealWallieWall Sheet i can't do that too a cr...
23835              fuck ray rice ,someone crack that bitch
Name: tweet, dtype: object
<class 'pandas.core.series.Series'>
11696    if your twitter is private but your instagram ...
4507          username sheet i can t do that too a cripple
23835              fuck ray rice  someone crack that bitch
Name: tweet, dtype: object
X-----------> if your twitter is private but your instagram is public u r probs a trash talking thot
{'your': 1, 'is': 2, 'if': 3, 'twitter': 4, 'private': 5, 'but': 6, 'instagram': 7, 'public': 8, 'u': 9, 'r': 10, 'probs': 11, 'a': 12, 'trash': 13, 'talking': 14, 'thot': 15}
Number of unique words in dictionary= 15
Dictionary is =  {'your': 1, 'is': 2, 'if': 3, 'twitter': 4, 'private': 5, 'but': 6, 'instagram': 7, 'public': 8, 'u': 9, 'r': 10, 'probs': 11, 'a': 12, 'trash': 13, 'talking': 14, 'thot': 15}
Dense vec

In [35]:
print(data)

11696    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
4507     [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
23835    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Name: tweet, dtype: object
